In [1]:
from diffusers import DDPMScheduler
import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.utils.data import Dataset, DataLoader
import torchaudio
from tqdm import tqdm
import pandas as pd
import os

from VQ_model import get_model
from diffusion_model import VQ_diffwave
import params as params

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vq_model = get_model(data_variance = None, inference = True)
vq_model.load_state_dict(torch.load('VQ_pth/model.pth'))

<All keys matched successfully>

In [18]:
with torch.no_grad():
    wav, sr = torchaudio.load('../dataset/ESC-50-master/audio/1-103298-A-9.wav')
    wav = torchaudio.transforms.Resample(sr, 16000)(wav)
    wav = wav.unsqueeze(1)
    z = vq_model(wav)
    output = vq_model.decoder(z)
    sample = output.squeeze(0)
    print(sample.shape)
    sample_wav = torchaudio.save('sample.wav', sample.cpu(), 16000)

torch.Size([1, 80000])
